In [214]:
import pandas as p
import pickle
import numpy as np
from IPython.display import display
import re
features=['surface','Parking','prix','€/ mois','€ / m²','Pièces','Chambre','Année de construction','Indice d\'émission de gaz à effet de serre','Diagnostic de performance énergétique :','Ascenseur','Toilette','Cuisine','Salle de séjour','entrée','Cave','Vue','Gardien','Digicode','Salle de bain','Interphone',"Salle d\'eau",'Alarme','Calme','Parquet','Rangements','du prix de','Balcon','proprety_type','type','adresse','link']
p.set_option('display.max_colwidth', 777)


with open('All_houses.pickle','rb') as f:
    All_houses=pickle.load(f)

#load data
df = p.DataFrame(data=All_houses,columns=features)
#turn prix to numeric, clean symbols
df['prix'] = df['prix'].str.replace('[\s:€]*','')
df['prix'] = p.to_numeric(df['prix'],errors='coerce')

#clean column for further use
df['€ / m²']=df['€ / m²'].str.replace(' ','')
df['€ / m²']=p.to_numeric(df['€ / m²'],errors='coerce')

#get rid of useless spaces
df['€/ mois']=df['€/ mois'].str.replace(' ','')
df['€/ mois']=p.to_numeric(df['€/ mois'],errors='coerce')

#increase decrease of price 
df['du prix de']=df['du prix de'].str.replace('Baisse','-')
df['du prix de']=df['du prix de'].str.replace('[\s€(Hausse)]','')
df['du prix de']=p.to_numeric(df['du prix de'],errors="coerce")

#get rid of hanging s
d=df[['Parking','Chambre','Pièces','Toilette','Salle de bain','Interphone','Salle d\'eau','Alarme','Calme','Parquet','Rangements','Balcon','entrée','Gardien']]
d=d.apply(lambda x:x.str.replace('s',''))
d=d.apply(lambda x:x.str.replace('éparée','1'))

df[['Parking','Chambre','Pièces','Toilette','Salle de bain','Interphone','Salle d\'eau','Alarme','Calme','Parquet','Rangements','Balcon','entrée','Gardien']]=d

#replace entrée
df['entrée']=df['entrée'].str.replace('Une','1')
df['entrée']=df['entrée'].fillna(1)

#replace cave salle de sejour
df[['Cave','Salle de séjour']]=df[['Cave','Salle de séjour']].apply(lambda x: x.str.replace(':*\s*\d*m²','1',regex=True))
df['entrée']=df['entrée'].str.replace('Une','1')
df['entrée']=df['entrée'].fillna(1)

#cleansing indices 
d=df[['Indice d\'émission de gaz à effet de serre','Diagnostic de performance énergétique :']]
df[list(d.columns)]=d.apply(lambda x: x.replace('[\s\w]*DPE[\s\w]*',np.NaN,regex=True))

#ordinal data
df['Cuisine']=df['Cuisine'].replace({'aucune':0,'coin cuisine':1,'équipée':2,'séparée équipée':2,'séparée':2,'américaine':3,'américaine équipée':4,'aménagée séparée':3})

#1 pieces pour les studios
df.loc[df.type=='Studio',['Pièces']]=1
#0 chambre studio 
df.loc[df.type=='Studio',['Chambre']]=0
#drop empty prix surface & more 
df.dropna(subset=['prix','surface','Chambre','Pièces'],inplace=True)

df.drop('Salle d\'eau',axis=1,inplace=True)
df['Toilette'].fillna(1,inplace=True)


#fix surface values
df['surface']=df['prix']/df['€ / m²']

df[df.adresse==""]
#remove no adresse tuple
df.drop([1361],inplace=True)
# fill nan with 0
df.fillna(0,inplace=True)
#type for trainnig
df['proprety_type']=df['type'].replace({'Studio':0,'Appartement':1,'maison':2})
#fix index
df.index=range(len(df))

lat_long=p.read_csv('lat_long.csv')
df.insert(0,'long',lat_long[' long'])
df.insert(0,'lat',lat_long['lat'])
#convert to numeric
df.iloc[:,0:-3]=p.to_numeric(df.iloc[:,0:-3].stack(),errors='coerce').unstack()
df.dropna(inplace=True)
df.index=range(len(df))

In [ ]:
with open('complete_dataframe.pickle','wb') as file:
    pickle.dump(df,file)

,lat,long,surface,Parking,prix,€/ mois,€ / m²,Pièces,Chambre,Année de construction,...,Alarme,Calme,Parquet,Rangements,du prix de,Balcon,proprety_type,type,adresse,link
0,48.886946,2.367851,93.004307,0.0,799000.0,3301.0,8591.0,4.0,3.0,0.0,...,0.0,0.0,0.0,0.0,-31000.0,0.0,1.0,Appartement,Paris 19ème,"https://www.seloger.com/annonces/achat/appartement/paris-19eme-75/155003819.htm?projects=2&types=1,2&places=[{ci:750056}]&enterprise=0&qsVersion=1.0&bd=ListToDetail"
1,48.833742,2.322970,40.278925,0.0,439000.0,1818.0,10899.0,2.0,1.0,1930.0,...,0.0,1.0,0.0,0.0,-13000.0,0.0,1.0,Appartement,Paris 14ème,"https://www.seloger.com/annonces/achat/appartement/paris-14eme-75/jean-moulin-porte-d-orleans/160490815.htm?projects=2&types=1,2&places=[{ci:750056}]&enterprise=0&qsVersion=1.0&bd=ListToDetail"
2,48.892129,2.348956,42.000158,0.0,530000.0,2193.0,12619.0,2.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,Appartement,Paris 18ème,"https://www.seloger.com/annonces/achat/appartement/paris-18eme-75/goutte-d-or-chateau-rouge/159029769.htm?projects=2,5&types=1,2&natures=1,2,4&places=[{div:2238}]&enterprise=0&qsVersion=1.0&bd=ListToDetail"
3,48.989169,2.319711,134.015595,2.0,550000.0,2275.0,4104.0,5.0,3.0,2020.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,maison,Montmorency,"https://www.seloger.com/annonces/achat/maison/montmorency-95/centre-montmorency-centre/160348801.htm?projects=2,5&types=1,2&natures=1,2,4&places=[{div:2238}]&enterprise=0&qsVersion=1.0&bd=ListToDetail"
4,48.873930,2.387481,28.471412,0.0,244000.0,1014.0,8570.0,2.0,1.0,1900.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Appartement,Paris 20ème,"https://www.seloger.com/annonces/achat/appartement/paris-20eme-75/telegraphe-pelleport-saint-fargeau/157752687.htm?projects=2,5&types=1,2&natures=1,2,4&places=[{div:2238}]&enterprise=0&qsVersion=1.0&bd=ListToDetail"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11877,48.884683,2.269566,58.737781,1.0,685000.0,2832.0,11662.0,3.0,2.0,1960.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Appartement,Neuilly-sur-Seine,"https://www.seloger.com/annonces/achat/appartement/neuilly-sur-seine-92/ile-de-la-jatte-parc-d-orleans/160893893.htm?projects=2&types=1,2&natures=1,2&places=[{cp:92}|{cp:93}|{cp:91}|{cp:94}]&enterprise=0&qsVersion=1.0&bd=ListToDetail"
11878,48.835665,2.240206,77.999257,0.0,630000.0,2605.0,8077.0,5.0,3.0,1960.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,Appartement,Boulogne-Billancourt,"https://www.seloger.com/annonces/achat/appartement/boulogne-billancourt-92/grenier-point-du-jour/155053863.htm?projects=2&types=1,2&natures=1,2&places=[{cp:92}|{cp:93}|{cp:91}|{cp:94}]&enterprise=0&qsVersion=1.0&bd=ListToDetail"
11879,48.808691,2.188349,73.995772,0.0,560000.0,2317.0,7568.0,4.0,2.0,1930.0,...,0.0,0.0,1.0,1.0,0.0,0.0,2.0,maison,Chaville,"https://www.seloger.com/annonces/achat/maison/chaville-92/158758885.htm?projects=2&types=1,2&natures=1,2&places=[{cp:92}|{cp:93}|{cp:91}|{cp:94}]&enterprise=0&qsVersion=1.0&bd=ListToDetail"
11880,48.910595,2.289045,80.005527,1.0,579000.0,2395.0,7237.0,4.0,3.0,2006.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Appartement,Asnières-sur-Seine,"https://www.seloger.com/annonces/achat/appartement/asnieres-sur-seine-92/160819305.htm?projects=2&types=1,2&natures=1,2&places=[{cp:92}|{cp:93}|{cp:91}|{cp:94}]&enterprise=0&qsVersion=1.0&bd=ListToDetail"


#### df.dtypes